# Model 생성과 Migration

DB 에 해당 하는 Model 의 생성 방법과 이를 어플리케이션에서 임포트 하여 사용하기 위한 준비과정을 살펴 본다.

[APP 폴더구조]
- admin.py      - 관리자 페이지 생성시 관리 대상을 설정 할 수 있는 파일
- apps.py       - 어플리케이션의 이름과 같은 기본정인 정보를 관리하는 파일
- migrations 
- models.py     - ORM 형식의 DB 관리정보를 관리하는 파일
- tests.py
- urls.py       - 접속 가능한 서브도메인과 페이지 정보를 관리하는 파일
- views.py

In [30]:
# polls 앱내 파일 확인
!ls ../polls/

__init__.py
__pycache__
admin.py
apps.py
migrations
models.py
tests.py
urls.py
views.py


## 1. Model 생성
APP내 models.py 파일을 통해 해당 앱에서 사용하고자 하는 모델 속성정보와 CASCADE 연결 정보 등을 정의 할 수 있다.

In [31]:
## model 파일을 수정 하여 ORM 정보를 등록한다.
!cat ../polls/models.py

from django.db import models

# decalred ORM Information (Object Relation Mapping)
class Question(models.Model):
    question_test = models.CharField(max_length=200)
    pub_data = models.DateTimeField('date published')
    
class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE) # FK �뿰寃�
    
    choice_text = models.CharField(max_length=200)
    vote = models.IntegerField(default=0)    


## 2. App 을 프로젝트에 연결하기

APP 내 정의한 Model 정보를 인식시키려면 프로젝트 단위의 Migrate 과정이 필요한데, 이를 위해서는 APP 를 프로젝트와 연결하는 작업이 선행 되어야 한다.

In [10]:
# 프로젝트에 App 을 연결 하기 위해 우선 App 의 이름을 확인한다.
!cat ../polls/apps.py

from django.apps import AppConfig


class PollsConfig(AppConfig):
    default_auto_field = 'django.db.models.BigAutoField'
    name = 'polls'


In [41]:
# App 이름을 /프로젝트/settins.py 에 등록한다.
!head -30 ../config/settings.py

# INSTALLED_APPS 내에 polls 를 추가
# 'polls.apps.PollsConfig', # adding my apps

from pathlib import Path

# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(__file__).resolve().parent.parent


# Quick-start development settings - unsuitable for production
# See https://docs.djangoproject.com/en/3.2/howto/deployment/checklist/

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = 'django-insecure-^-o$e^lif^tcq08r2&ubj20j91!j77o-)j6!nfrl+l5w8vd=l^'

# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = True

ALLOWED_HOSTS = []


# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'polls.apps.PollsConfig', # adding my apps
]



## 3. App 에 존재하는 Models 정보를 Migrations 시키기
App 내 Model 정보를 읽어 Migrations 파일을 생성하도록 지시한다.

In [32]:
# makemigrations 명령어를 통하여 polls 앱의 models 정보를 migration 시킨다.
!cd.. & python manage.py makemigrations polls

No changes detected in app 'polls'


In [35]:
# 마이그레이션은 models 정보가 변경될 때 마다 추가적으로 생성 가능하다.
!ls ../polls/migrations/

# 0001_initial.py 은 히스토리 버전으로 추가 변경 사항이 발생 할 때 마다 일련번호가 증가된 파일이 생성된다.

0001_initial.py
__init__.py
__pycache__


In [39]:
# /polls.migrations/ 폴더에 존재하는 initial.py 내용을 확인
!head -30 ../polls/migrations/0001_initial.py


# [ID | QUESTION | DATA]
# 로 구성된 Question 테이블

# [ID | CHOICE_TEXT | VOTE | QUESTION(FK)]
# 로 구성된 CHoice 테이블

# 의 모델 정보가 각 생성되어져 있다.

# Generated by Django 3.2.12 on 2022-02-05 12:10

from django.db import migrations, models
import django.db.models.deletion


class Migration(migrations.Migration):

    initial = True

    dependencies = [
    ]

    operations = [
        migrations.CreateModel(
            name='Question',
            fields=[
                ('id', models.BigAutoField(auto_created=True, primary_key=True, serialize=False, verbose_name='ID')),
                ('question_test', models.CharField(max_length=200)),
                ('pub_data', models.DateTimeField(verbose_name='date published')),
            ],
        ),
        migrations.CreateModel(
            name='Choice',
            fields=[
                ('id', models.BigAutoField(auto_created=True, primary_key=True, serialize=False, verbose_name='ID')),
                ('choice_text', models.CharField(max_length=200)),
                ('vote', models.IntegerField(default=0)),
                ('question', models.ForeignKey(on_delete=django.db

## 4. 프로젝트 마이그레이트 시키기
앞서 진행한 Migrations 작업은 Model 의 테이블 정보의 변경이나 생성정보를 관리하는 파일생성이 주 목적이다.  
Migrate 작업을 통해 실제 DB에 변경내용을 적용시키는 단계라 보면 쉽다.

In [28]:
# 프로젝트를 기준으로 migrate 작업을 실행한다.
!cd.. && python manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, polls, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_nam

## 5. Makemigrations 와 Migrate 의 차이

- makemigration : 모델의 변경 사항을 파일 형태로 생성.
- migrate : 생성된 Migrations 파일을 근거로 DB 모델을 변경 처리.

In [29]:
# Migrate 가 진행되는 SQL 작업을 아래의 명령어로 확인 할 수 있다.
!cd.. && python manage.py sqlmigrate polls 0001

BEGIN;
--
-- Create model Question
--
CREATE TABLE "polls_question" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "question_test" varchar(200) NOT NULL, "pub_data" datetime NOT NULL);
--
-- Create model Choice
--
CREATE TABLE "polls_choice" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "choice_text" varchar(200) NOT NULL, "vote" integer NOT NULL, "question_id" bigint NOT NULL REFERENCES "polls_question" ("id") DEFERRABLE INITIALLY DEFERRED);
CREATE INDEX "polls_choice_question_id_c5b4b260" ON "polls_choice" ("question_id");
COMMIT;
